In [86]:
%load_ext autoreload
%autoreload

from data.models import Coin
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
Coin.update(symbol='BTC', ccy='KRW', exchange='upbit')

2019-04-05 21:00:00 - 2019-06-28 04:00:00+00:00
2019-01-12 13:00:00 - 2019-04-05 20:59:59
2018-10-21 05:00:00 - 2019-01-12 12:59:59
2018-07-29 21:00:00 - 2018-10-21 04:59:59
2018-05-07 13:00:00 - 2018-07-29 20:59:59
2018-04-27 22:00:00 - 2018-05-07 12:59:59
completed


In [78]:
class Master:
    def __init__(self):
        pass
    
    @classmethod
    def inherited_from(cls, parent_cls):
        if parent_cls in cls.__mro__:
            return True
        else:
            raise cls.NotInheritedError()
            
    class NotInheritedError(Exception):
        pass

In [123]:
class Data(Master):
    def __init__(self):
        pass
    
    @property
    def _data_type(self):
        return None
    
    
class MarketData(Data):
    def price(self, asof=None, start=None, end=None, tickers=None):
        return self.get_marketdata(asof=asof, start=start, end=end, tickers=tickers, item='price')
    
    def _marketdata(self, *, start, end, items=[]):
        raise NotImplementedError
    

class CoinMarketData(MarketData):
    def __init__(self, *, symbol, ccy, exchange):
        Coin.update(symbol=symbol, ccy=ccy, exchange=exchange)
        self.coin = Coin.objects.filter(symbol=symbol, ccy=ccy, exchange=exchange)
        super().__init__()
        
    def _marketdata(self, *, start, end, items=[]):
        marketdata = self.coin.filter(time__gte=start, time__lte=end)
        return pd.DataFrame(list(marketdata.values('time', *items))).set_index('time').sort_index()

In [124]:
data = CoinMarketData(symbol='BTC', ccy='KRW', exchange='upbit')

fully updated already


In [125]:
data._marketdata(start='2019-06-28 00:00:00+00:00', end='2019-06-29 00:00:00+00:00', items=['close', 'volume'])

,close,volume
time,,
2019-06-28 00:00:00+00:00,14401000.0,375.83
2019-06-28 01:00:00+00:00,14196000.0,310.37
2019-06-28 02:00:00+00:00,13758000.0,312.92
2019-06-28 03:00:00+00:00,14067000.0,480.83
2019-06-28 04:00:00+00:00,13864000.0,413.52
2019-06-28 05:00:00+00:00,13898000.0,382.33
